### Pydantic workflow - Basics

Pydantic models are the foundations of data validations in Python. They use Python type annotations to define the structure and validate data at runtime. Here's a detailed exploration of basic model creation with several examples

In [2]:
from pydantic import BaseModel


In [3]:
class Person(BaseModel):
    name: str
    age: int
    city: str

person = Person(name="John", age=30, city="New York")
print(person)


name='John' age=30 city='New York'


In [4]:
type(person)

__main__.Person

## Now see if we have wrong type, we get an issue

In [5]:
person_1 = Person(name="John", age="30", city=12)
print(person_1)

ValidationError: 1 validation error for Person
city
  Input should be a valid string [type=string_type, input_value=12, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

So, the pydantic's BaseModel is having the validation as needed, suppose we have the normal DataClass decorator. We see that the error is not popping meaning we have no DATA VALIDATION

In [7]:
from dataclasses import dataclass
@dataclass
class Person:
    name: str
    age: int
    city: str

person_2 = Person(name="John", age="30", city=12)

In [8]:
print(person_2)

Person(name='John', age='30', city=12)


The output should cause an issue, but it's not. This led to the use of Pydantic.
The data validation is so fast, because the backend we are using is RUST.

### Model with option field

In [14]:
from typing import Optional
from typing import List

class Employee(BaseModel):
    name: str
    age: int
    salary: Optional[float] = None
    is_active: Optional[bool] = True
    skills: Optional[List[str]] = None

emp = Employee(name="John", age=30)
print(emp)


name='John' age=30 salary=None is_active=True skills=None


Type casting is possible !

In [18]:
try:
    emp_1 = Employee(name="John", age="30", salary="100000")
except Exception as e:
    print(e)

# no exception, because of type coercion

In [17]:
try:
    emp_1 = Employee(name="John", age="abc", salary="100000")
except Exception as e:
    print(e)

1 validation error for Employee
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='abc', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


### Create complex structures with nested models


In [23]:
class Address(BaseModel):
    street: str
    city: str
    zip_code: int

class Company(BaseModel):
    name: str
    address: Address

company = Company(name="Tech Corp", address={"street":"123 Main St", "city":"Anytown", "zip_code":"12345"})
print(company)


name='Tech Corp' address=Address(street='123 Main St', city='Anytown', zip_code=12345)


### Pydantic Fields: Customization and Constraints

The Field function in Pydantic enhances model fields beyond basic type hits by allowing you to specity validation rules, default values, aliases and more. Here's a comprehensive tutorial with examples.

In [24]:
from pydantic import BaseModel, Field

# A class with custom validation rules
class Item(BaseModel):
    name:str=Field(min_length=3, max_length=15)
    price:float=Field(ge=0, le=1000) # greater than 0 and less than 1000
    quantity:int=Field(default=1, ge=1, le=100) # default is 1, greater than 1 and less than 100

In [26]:
print(Item.model_json_schema())
    

{'properties': {'name': {'maxLength': 15, 'minLength': 3, 'title': 'Name', 'type': 'string'}, 'price': {'maximum': 1000, 'minimum': 0, 'title': 'Price', 'type': 'number'}, 'quantity': {'default': 1, 'maximum': 100, 'minimum': 1, 'title': 'Quantity', 'type': 'integer'}}, 'required': ['name', 'price'], 'title': 'Item', 'type': 'object'}
